# Implémentation de méthodes élémentaires pour la classification supervisée : ADL, ADQ, NB et $k$-NN 


### *Adrien BERSIER, Thomas FRIEDLAND, Nicolas HUBERT*

Le jeu de données [Vertebral Column](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column) permet d'étudier les pathologies d'hernie discale et de Spondylolisthesis. Ces deux pathologies sont regroupées dans le jeu de données en une seule seule catégorie dite `Abnormale`. 

Il s'agit donc d'un problème de classification supervisée à deux classes :
- Normale (NO) 
- Abnormale (AB)    

avec 6 variables bio-mécaniques disponibles (features).

L'objectif du TP est d'implémenter quelques méthodes simples de classification supervisée pour ce problème.

Les données sont disponibles dans le fichier `column_2C.dat`  : lien [Data Folder](https://archive.ics.uci.edu/ml/machine-learning-databases/00212/).



In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.covariance import empirical_covariance
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

# Importation des données

> Télécharger le fichier column_2C.dat depuis le site de l'UCI : https://archive.ics.uci.edu/ml/datasets/Vertebral+Column#, par exemple avec la librairie [pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html). 

In [2]:
file_path= "C:/Users/acer/Documents/GitHub/Centrale-S2D/vertebral_column_data/column_2C.dat"
Vertebral = pd.read_table(file_path,
                          delim_whitespace=True
                         )
Vertebral.columns = ["pelvic_incidence",
                                       "pelvic_tilt",
                                       "lumbar_lordosis_angle",
                                       "sacral_slope",
                                       "pelvic_radius",
                                       "degree_spondylolisthesis",
                                       "class"]

> Vérifier à l'aide des méthodes `.head()`  et `describe()` que les données sont bien importées.

In [3]:
Vertebral.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,39.06,10.06,25.02,29.00,114.41,4.56,AB
1,68.83,22.22,50.09,46.61,105.99,-3.53,AB
2,69.30,24.65,44.31,44.64,101.87,11.21,AB
3,49.71,9.65,28.32,40.06,108.17,7.92,AB
4,40.25,13.92,25.12,26.33,130.33,2.23,AB


In [4]:
Vertebral.describe()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
count,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000
mean,60.488285,17.526699,51.970583,42.961877,117.982848,26.382654
std,17.263462,10.020300,18.570553,13.443779,13.293909,37.589284
min,26.150000,-6.550000,14.000000,13.370000,70.080000,-11.060000
25%,46.430000,10.660000,37.000000,33.340000,110.710000,1.630000
50%,58.600000,16.300000,49.780000,42.440000,118.340000,12.070000
75%,72.960000,21.940000,63.000000,52.740000,125.480000,41.420000
max,129.830000,49.430000,125.740000,121.430000,163.070000,418.540000


> Les librairies de ML telles que `sckitlearn` prennent en entrée des tableau numpy (pas du pandas). Créer un tableau numpy pour les features et une liste pour les classes : que vous nommerez `VertebralVar`.

In [5]:
VertebralVar_pandas = Vertebral.iloc[:,0:6] # ne prend pas la 6ème colonne
# extraction de colonnes en pandas
# puis conversion en tableau numpy :
VertebralVar  = VertebralVar_pandas.values # X observés
VertebralClas = Vertebral.iloc[:,6].values # Y observé (dernière colonne)

# Découpage train / test

> En utilisant la fonction [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) de la librairie [`sklearn.model_selection`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection), sélectionner aléatoirement 200 observations pour l'échantillon d'apprentissage et garder le reste pour l'échantillon de validation. 

In [6]:
from sklearn.model_selection import train_test_split

VertebralVar_train,VertebralVar_test,VertebralClas_train, VertebralClas_test = train_test_split(VertebralVar,
                                                                                                VertebralClas, 
                                                                                                train_size=200, random_state=0)
ntot=VertebralClas.shape[0]
ntrain=VertebralVar_train.shape[0]
ntest=ntot-ntrain
ntrain

C:\Users\acer\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


200

In [7]:
VertebralVar_train

array([[ 50.91,  23.02,  47.  ,  27.9 , 117.42,  -2.53],
       [ 35.88,   1.11,  43.46,  34.77, 126.92,  -1.63],
       [ 68.83,  22.22,  50.09,  46.61, 105.99,  -3.53],
       ...,
       [ 65.54,  24.16,  45.78,  41.38, 136.44,  16.38],
       [ 40.56,  17.98,  34.  ,  22.58, 121.05,  -1.54],
       [ 50.83,   9.06,  56.3 ,  41.76,  79.  ,  23.04]])

In [8]:
VertebralClas_train

array(['AB', 'NO', 'AB', 'NO', 'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'NO', 'AB', 'AB',
       'AB', 'NO', 'AB', 'NO', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'NO', 'AB', 'AB', 'NO', 'NO', 'NO', 'NO', 'AB',
       'NO', 'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO',
       'NO', 'AB', 'NO', 'NO', 'NO', 'AB', 'AB', 'AB', 'NO', 'AB', 'NO',
       'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'NO',
       'AB', 'NO', 'AB', 'AB', 'NO', 'AB', 'AB', 'NO', 'AB', 'NO', 'AB',
       'AB', 'AB', 'NO', 'NO', 'AB', 'AB', 'AB', 'NO', 'AB', 'NO', 'AB',
       'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'NO', 'NO',
       'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'NO',
       'AB', 'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'NO',
       'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'NO', 'N

Remarque : on peut aussi le faire à la main avec la fonction [`sklearn.utils.shuffle`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html).

# Extraction des deux classes

> Extraire les deux sous échantillons Abnormale et Normale pour les données d'apprentissage et de test.

In [9]:
VertebralVar_train_AB = VertebralVar_train[VertebralClas_train=="AB"]
VertebralVar_train_NO = VertebralVar_train[VertebralClas_train=="NO"]
print(VertebralVar_train_AB)

[[ 5.0910e+01  2.3020e+01  4.7000e+01  2.7900e+01  1.1742e+02 -2.5300e+00]
 [ 6.8830e+01  2.2220e+01  5.0090e+01  4.6610e+01  1.0599e+02 -3.5300e+00]
 [ 4.2020e+01 -6.5500e+00  6.7900e+01  4.8580e+01  1.1159e+02  2.7340e+01]
 [ 7.2340e+01  1.6420e+01  5.9870e+01  5.5920e+01  7.0080e+01  1.2070e+01]
 [ 5.0070e+01  9.1200e+00  3.2170e+01  4.0950e+01  9.9710e+01  2.6770e+01]
 [ 4.7660e+01  1.3280e+01  3.6680e+01  3.4380e+01  9.8250e+01  6.2700e+00]
 [ 5.7290e+01  1.5150e+01  6.4000e+01  4.2140e+01  1.1674e+02  3.0340e+01]
 [ 4.1730e+01  1.2250e+01  3.0120e+01  2.9480e+01  1.1659e+02 -1.2400e+00]
 [ 3.7900e+01  4.4800e+00  2.4710e+01  3.3420e+01  1.5785e+02  3.3610e+01]
 [ 7.7110e+01  3.0470e+01  6.9480e+01  4.6640e+01  1.1215e+02  7.0760e+01]
 [ 6.7260e+01  7.1900e+00  5.1700e+01  6.0070e+01  9.7800e+01  4.2140e+01]
 [ 4.6860e+01  1.5350e+01  3.8000e+01  3.1500e+01  1.1625e+02  1.6600e+00]
 [ 8.4970e+01  3.3020e+01  6.0860e+01  5.1950e+01  1.2566e+02  7.4330e+01]
 [ 6.6800e+01  1.4550e+01

In [10]:
n_AB = VertebralVar_train_AB.shape[0]
n_NO = VertebralVar_train_NO.shape[0]
print(n_AB)
print(n_NO)

135
65


In [11]:
# from sklearn.model_selection import train_test_split
# ntot = Vertebral.shape[0]
# shuffled_indices = np.random.permutation(ntot)
# print(shuffled_indices)

In [12]:
# ntrain = shuffled_indices[:200]
# ntest = shuffled_indices[200:]
# ntrain.shape

In [13]:
# VertebralVar_train,VertebralVar_test,VertebralClas_train, VertebralClas_test = train_test_split(VertebralVar,
#                                                                                                 VertebralClas, 
#                                                                                                 train_size=ntrain.shape[0], random_state=0)
# VertebralVar_train

# Analyse Discriminante Linéaire (LDA)

> Implémenter le classifieur de l'analyse discriminante linéaire en l'appliquant sur les données. Vous pourrez utiliser les fonctions [`empirical_covariance`](http://scikit-learn.org/stable/modules/generated/sklearn.covariance.EmpiricalCovariance.html#sklearn.covariance.EmpiricalCovariance) de  la librairie5 [`sklearn.covariance`](http://scikit-learn.org/stable/modules/covariance.html) et la fonction [`multivariate_normal`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html) de la librairie 
[`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html). Ajuster le classifieur sur l'échantillon d'apprentissage.

D'abord on calcule le centre et la matrice de covariance pour chacun des deux groupes  :

In [14]:
from sklearn.covariance import empirical_covariance
Cov_AB = empirical_covariance(VertebralVar_train_AB)
Cov_NO = empirical_covariance(VertebralVar_train_NO)
mean_AB = VertebralVar_train_AB.mean(axis=0)
mean_NO = VertebralVar_train_NO.mean(axis=0)

In [15]:
print(mean_AB)
print(mean_NO)

[ 64.28807407  19.54296296  53.86251852  44.74555556 115.26918519
  36.82696296]
[ 52.83353846  13.01323077  44.48969231  39.81984615 122.94692308
   2.05215385]


Puis on calcule la matrice de covariance intra :

In [16]:
Intra_Cov = (n_AB * Cov_AB + n_NO * Cov_NO) / (n_AB + n_NO)
print(Intra_Cov)

[[ 258.23006193   77.26587814  193.30591263  180.9598814   -47.75392602
   351.46951707]
 [  77.26587814   84.53376818   56.75127579   -7.26833095   14.55822686
    89.08490231]
 [ 193.30591263   56.75127579  317.17205969  136.55449554  -15.45547742
   277.48800238]
 [ 180.9598814    -7.26833095  136.55449554  188.22438116  -62.3074406
   262.390221  ]
 [ -47.75392602   14.55822686  -15.45547742  -62.3074406   162.22365997
    41.90537798]
 [ 351.46951707   89.08490231  277.48800238  262.390221     41.90537798
  1404.17856577]]


On peut maintenant calculer le score sur chaque groupe pour chaque element des données de test  :
<center>
[Vraisemblance du groupe]  * [proba a priori ]

In [17]:
score_LDA_test = [   [ 
n_AB * multivariate_normal.pdf(x,mean=mean_AB, cov=Intra_Cov), # densité d'une gausienne multivariée, de moyenne mean_AB
    # et de matrice de var-covar Intra_Cov
n_NO * multivariate_normal.pdf(x,mean=mean_NO, cov=Intra_Cov),]  for x in VertebralVar_test] 

# en mettant la même matrice de variance covariance, on fait du LDA, et non du QDA
score_LDA_test

[[6.049766206118149e-06, 5.221931243850173e-06],
 [4.843275403936803e-05, 5.6702274810849085e-05],
 [1.9817486689167988e-06, 4.5272598230228606e-08],
 [3.925778665287586e-06, 3.5902944146402e-07],
 [2.186403590189722e-07, 4.365144933236242e-09],
 [5.039711257700715e-05, 2.6712126703061875e-06],
 [2.4581812235795424e-05, 1.258279463157992e-05],
 [4.4480341736296455e-05, 2.120690725826617e-05],
 [3.045534948057192e-05, 4.574462716112964e-06],
 [2.99250345939698e-05, 7.285904365056379e-07],
 [3.7446410809425316e-05, 2.0355187338259323e-05],
 [0.00012918278687227416, 3.568692257117976e-05],
 [2.724312120379971e-05, 1.1370043278801602e-06],
 [4.1723227830175604e-05, 2.627054531696764e-05],
 [5.8240651651899696e-05, 4.8745501715579857e-05],
 [1.2418603231274043e-05, 1.0498418692751563e-05],
 [1.0337085789689213e-05, 3.403371465357874e-05],
 [4.280959588443644e-06, 5.028467514955806e-06],
 [1.7675977682070975e-05, 3.5861454502979453e-06],
 [1.8207316613703397e-06, 1.2896662185392155e-05],
 [6

et on applique la règle de decision finale sur les données de test : max a posteriori :

In [18]:
pred_LDA_test = [l.index(max(l)) for l in score_LDA_test]
pred_LDA_test
# VertebralClas_test  # pour comparaison

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0]

> Evaluer les performances de la méthode sur l'échantillon test. Vous pourrez utiliser la fonction [`confusion_matrix`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix) de la librairie [`sklearn.metrics`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [19]:
from sklearn.metrics import confusion_matrix
vraie_classe_test = 1 * (VertebralClas_test == "NO") 
cnf_matrix_LDA_test = confusion_matrix(vraie_classe_test,pred_LDA_test)
cnf_matrix_LDA_test.astype('float') / cnf_matrix_LDA_test.sum(axis=1).reshape(-1,1) 

array([[0.98648649, 0.01351351],
       [0.37142857, 0.62857143]])

> Vérifier que l'évaluation de la méthode sur les données d'apprentissage donne un résulat sensiblement plus optimiste.

In [20]:
score_LDA_train =  [   [ 
n_AB * multivariate_normal.pdf(x,mean=mean_AB, cov=Intra_Cov), # densité d'une gausienne multivariée, de moyenne mean_AB
    # et de matrice de var-covar Intra_Cov
n_NO * multivariate_normal.pdf(x,mean=mean_NO, cov=Intra_Cov),]  for x in VertebralVar_train] 
pred_LDA_train = [l.index(max(l)) for l in score_LDA_train]
vraie_classe_train = 1 * (VertebralClas_train == "NO") 
cnf_matrix_LDA_train =  confusion_matrix(vraie_classe_train,pred_LDA_train)
cnf_matrix_LDA_train.astype('float') / cnf_matrix_LDA_train.sum(axis=1).reshape(-1,1) 

array([[0.91851852, 0.08148148],
       [0.4       , 0.6       ]])

Il existe bien sûr une fonction scikit-learn pour la méthode LDA : voir [ici](http://scikit-learn.org/0.16/modules/generated/sklearn.lda.LDA.html).

# Analyse Discriminante Quadratique (QDA)

> Implémenter de même pour la méthode de l'analyse Discriminante Quadratique (QDA)

In [21]:
score_QDA_test =  [   [ 
n_AB * multivariate_normal.pdf(x,mean=mean_AB, cov=Cov_AB), # densité d'une gausienne multivariée, de moyenne mean_AB
    # et de matrice de var-covar Intra_Cov
n_NO * multivariate_normal.pdf(x,mean=mean_NO, cov=Cov_NO),]  for x in VertebralVar_test] 
pred_QDA_test = [l.index(max(l)) for l in score_QDA_test]
cnf_matrix_QDA_test = confusion_matrix(vraie_classe_test,pred_QDA_test)
cnf_matrix_QDA_test.astype('float') / cnf_matrix_QDA_test.sum(axis=1).reshape(-1,1) 

array([[0.86486486, 0.13513514],
       [0.14285714, 0.85714286]])

Il existe bien sûr une fonction scikit-learn pour la méthode QDA : voir [ici](http://scikit-learn.org/0.16/modules/generated/sklearn.qda.QDA.html).

# Gaussian Naive Bayes

> Implémenter le classifieur naif bayesien et évaluer les performances de la méthode sur l'échantillon test.

Calcul des variances de chaque variable pour chacun des deux groupes :

In [22]:
VertebralVar_test_AB = VertebralVar_test[VertebralClas_test=="AB"]
VertebralVar_test_NO = VertebralVar_test[VertebralClas_test=="NO"]
print(VertebralVar_test_AB)

[[ 76.15  21.94  82.96  54.21 123.93  10.43]
 [ 95.38  24.82  95.16  70.56  89.31  57.66]
 [ 92.03  35.39  77.42  56.63 115.72  58.06]
 [ 43.72   9.81  52.    33.91  88.43  40.88]
 [ 85.58  30.46  78.23  55.12 114.87  68.38]
 [ 78.49  22.18  60.    56.31 118.53  27.38]
 [ 43.58  16.51  47.    27.07 109.27   8.99]
 [ 75.65  19.34  64.15  56.31  95.9   69.55]
 [ 70.95  20.16  62.86  50.79 116.18  32.52]
 [ 83.4   34.31  78.42  49.09 110.47  49.67]
 [ 43.35   7.47  28.07  35.88 112.78   5.75]
 [ 45.54  13.07  30.3   32.47 117.98  -4.99]
 [ 48.03   3.97  58.34  44.06 125.35  35.  ]
 [ 82.41  29.28  77.05  53.13 117.04  62.77]
 [ 63.83  20.36  54.55  43.47 112.31  -0.62]
 [ 81.66  28.75  58.23  52.91 114.77  30.61]
 [ 89.68  32.7   83.13  56.98 129.96  92.03]
 [ 53.57  20.46  33.1   33.11 110.97   7.04]
 [ 70.22  39.82  68.12  30.4  148.53 145.38]
 [ 68.61  15.08  63.01  53.53 123.43  39.5 ]
 [ 88.62  29.09  47.56  59.53 121.76  51.81]
 [ 48.26  16.42  36.33  31.84  94.88  28.34]
 [ 85.64  

In [23]:
mean_AB_test = VertebralVar_test_AB.mean(axis=0)
mean_NO_test = VertebralVar_test_NO.mean(axis=0)
print(mean_AB)

[ 64.28807407  19.54296296  53.86251852  44.74555556 115.26918519
  36.82696296]


In [33]:
# variances estimées coord par coord pour AB (sur le train) :
var_AB = np.var(VertebralVar_train_AB)
# variances estimées coord par coord pour NO (sur le train) :
var_NO = np.var(VertebralVar_train_NO)
print(var_NO)

1601.587013254438


In [36]:
# on forme les matrices de covariance (matrices diagonales car indep) :
Cov_NB_AB =  np.diag(np.diag(Cov_AB))
Cov_NB_NO =  np.diag(np.diag(Cov_NO))
Cov_NB_NO

array([[170.61558902,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  43.35198187,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        , 160.16678144,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,  97.48687844,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         78.0323413 ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  32.44572767]])

Calcul du "score" sur chaque groupe pour chaque element des données test : 
<center>
 [Vraisemblance du groupe]  * [proba a priori ]

In [48]:
score_NB_test = [   [ 
n_AB * multivariate_normal.pdf(x,mean=mean_AB_test, cov=Cov_NB_AB), # densité d'une gausienne multivariée, de moyenne mean_AB
    # et de matrice de var-covar Intra_Cov
n_NO * multivariate_normal.pdf(x,mean=mean_NO_test, cov=Cov_NB_NO),]  for x in VertebralVar_test] 
pred_NB_test = [l.index(max(l)) for l in score_NB_test] 

In [49]:
cnf_matrix_NB_test = confusion_matrix(vraie_classe_test,pred_NB_test)
cnf_matrix_NB_test.astype('float') / cnf_matrix_NB_test.sum(axis=1).reshape(-1,1) 

array([[0.77027027, 0.22972973],
       [0.08571429, 0.91428571]])

>  Il existe bien sûr une fonction scikit-learn  pour la méthode Naive Bayes : voir [ici](http://scikit-learn.org/stable/modules/naive_bayes.html). Vérifier que votre prédicteur donne la même réponse de cette fonction.

In [59]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(VertebralVar_test,VertebralClas_test)
y_pred=gnb.predict(VertebralVar_test)
print(y.shape, vraie_classe_test.shape)
cnf_matrix_NB_test_bis = confusion_matrix(vraie_classe_test,y_pred)
cnf_matrix_NB_test_bis.astype('float') / cnf_matrix_NB_test_bis.sum(axis=1).reshape(-1,1) 

(109,) (109,)


ValueError: Mix of label input types (string and number)

# Classifieur par plus proches voisins

Il est préférable d'utiliser la structure de données de type [k-d tree](https://en.wikipedia.org/wiki/K-d_tree) pour effectuer des requêtes de plus proches voisins dans un nuage de points. 

> Contruction du k-d tree pour les données train (pour la métrique euclidienne) :

In [63]:
from sklearn.neighbors import KDTree
from scipy import spatial
tree =  spatial.KDTree(VertebralVar_train, leafsize=2)

> Rechercher les 10 plus proches voisins dans les données d'apprentissage du premier point des données de test et afficher les classes de ces voisins observations.

In [88]:
indices_voisins =  tree.query(VertebralVar_train[:1,],k=10)[1]
print(indices_voisins) 

[[  0  45 146  14 154  51  68  19  39  58]]


In [97]:
classes_voisins = [VertebralClas_train[i,] for i in indices_voisins]
print(classes_voisins)  

[array(['AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'NO', 'NO'],
      dtype=object)]


Pour le classifieur par plus proches vosins, la prediction est la classe majoritaire des k plus proches voisins.

> Donner la prédiction pour le premier point de test par vote majoritaire sur ses 10 plus proches voisins 

In [101]:
def vote(classes_voisins):
    n_AB_kd=0
    n_NO_kd=0
    for i in range(len(classes_voisins)):
        if classes_voisins[0][i]=="AB" : n_AB_kd += 1
        else : n_NO_kd += 1
    if n_AB_kd > n_NO_kd:
        return("AB")
    else:
        return("NO")

print("Pour le premier point, le vote majoritaire est", vote(classes_voisins),)

Pour le premier point, le vote majoritaire est AB


> Donner la prediction du classifieur ppv pour toutes les données de test. Evaluer la qualité du classifieur.

In [102]:
def pred_classes_voisins(tree,var,clas,i,k):
    indices_voisins = tree.query(var[(i-1):i], k)[1]
    classes_voisins=clas[indices_voisins]
    return classes_voisins

tree_test=KDTree(VertebralVar_test, leaf_size=2)
vote(pred_classes_voisins(tree_test, VertebralVar_test, VertebralClas_test, 9, 10))

'AB'

In [ ]:
k_class = ### CHOISIR  ####  #nombre de plus proche voisins utilisés
pred_kNN_test =  ### TO DO ####
cnf_matrix_kNN =### TO DO ####
cnf_matrix_kNN.astype('float') / cnf_matrix_kNN.sum(axis=1).reshape(-1,1) 

Il existe bien sûr une fonction scikit-learn pour le classifieur plus proche voisin, voir [ici](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).